##### Work flow

1. New Data -> 
2. Data Preprocessing -> 
3. Train Test split -> 
4. Logistic Regression model -> 
5. Trained Logistic Regression model -> 
6. New Data  ------------------------> Real News or Fake news


1. id: unique id for a news article
2. title: the title of a news article
3. author: author of the news article
4. text: the text of the article; could be incomplete
5. label: a label that marks the article as potentially unreliable \
        * 1: unreliable\
        * 0: reliable 
* test.csv: A testing training dataset with all the same attributes at train.csv without the label.



In [4]:
# import libraries 
import numpy as np # used for creating numpy array 
import pandas as pd # used for creating dataframe.
import re # re is used for searching word in text or paragraph
from nltk.corpus import stopwords #They are words that you do not want to use to describe the topic of your content.
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer # used to convert text into feature vector
from sklearn.model_selection import train_test_split # used to split our dataset, training and test our data
from sklearn.linear_model import LogisticRegression # 
from sklearn.metrics import accuracy_score


In [5]:
# download stopwords 
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [6]:
print(stopwords.words('English')) # these stopwords does not add values to our dataset, we will remove them 

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [8]:
# loading the dataset 
news_dataset = pd.read_csv('./train.csv')
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [9]:
news_dataset.shape

(20800, 5)

In [10]:
# check number of missing values in the dataset
news_dataset.isna().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [33]:
# replacing null values with empty string 
news_dataset = news_dataset.fillna(' ')
news_dataset.isna().sum()

id          0
title       0
author      0
text        0
label       0
combined    0
dtype: int64

In [34]:
# combine author column and title column together 
news_dataset ['combined'] = news_dataset['author']+' '+news_dataset['title']

In [36]:
print(news_dataset['combined'])

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: combined, Length: 20800, dtype: object


In [35]:
# seperating the data and label
X =  news_dataset.drop(columns = 'label',axis = 1)
y = news_dataset['label']


In [22]:
print(X)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

In [37]:
print(y)

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 20800, dtype: int64


* Stemming: Steming is the process of reducing a word to its root word
* example: actor, actress, acting ------> act


In [24]:
port_stem = PorterStemmer()

In [38]:
# function to be used for stemming - 
def stemming(content_input):
    stemmed_content = re.sub('[^a-zA-Z]', '  ',content_input)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content =[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [39]:
# Apply the function to our columns 
news_dataset['combined'] = news_dataset['combined'].apply(stemming)

In [40]:
print(news_dataset['combined'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: combined, Length: 20800, dtype: object


In [41]:
# separeting the data and label
X = news_dataset['combined'].values
y = news_dataset['label'].values

In [43]:
print(X)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']


In [51]:
# previously, we learned that 0 = real news, and 1 =  fake news
print(y) # display labels 

[1 0 1 ... 0 1 1]


In [46]:
news_dataset.shape

(20800, 6)

In [47]:
# convert textural data into meaningful numeric data
vectorizer = TfidfVectorizer() # Tf - time frequency and Idf transfer inverse documents frequency; it does count a number of times a particular word is repeated in a document
vectorizer.fit(X) 

X = vectorizer.transform(X)

In [49]:
print(X)

  (0, 15686)	0.28485063562728646
  (0, 13473)	0.2565896679337957
  (0, 8909)	0.3635963806326075
  (0, 8630)	0.29212514087043684
  (0, 7692)	0.24785219520671603
  (0, 7005)	0.21874169089359144
  (0, 4973)	0.233316966909351
  (0, 3792)	0.2705332480845492
  (0, 3600)	0.3598939188262559
  (0, 2959)	0.2468450128533713
  (0, 2483)	0.3676519686797209
  (0, 267)	0.27010124977708766
  (1, 16799)	0.30071745655510157
  (1, 6816)	0.1904660198296849
  (1, 5503)	0.7143299355715573
  (1, 3568)	0.26373768806048464
  (1, 2813)	0.19094574062359204
  (1, 2223)	0.3827320386859759
  (1, 1894)	0.15521974226349364
  (1, 1497)	0.2939891562094648
  (2, 15611)	0.41544962664721613
  (2, 9620)	0.49351492943649944
  (2, 5968)	0.3474613386728292
  (2, 5389)	0.3866530551182615
  (2, 3103)	0.46097489583229645
  :	:
  (20797, 13122)	0.2482526352197606
  (20797, 12344)	0.27263457663336677
  (20797, 12138)	0.24778257724396507
  (20797, 10306)	0.08038079000566466
  (20797, 9588)	0.174553480255222
  (20797, 9518)	0.295420

* feed these data into machine learning model
* Splitting dataset into training and test data 

In [55]:
# Splitting our data into two data types X_train which will be 80% of the data and X_test will be 20% of the data 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify = y, random_state = 2)

In [56]:
# Train our model which is logistic regression model 
logist_model = LogisticRegression()

In [57]:
# train our model
logist_model.fit(X_train, y_train)

LogisticRegression()

* Evaluation of our model, accuracy score

In [59]:
# accuracy score on the train data 
X_train_pred = logist_model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_pred, y_train)

In [60]:
print(' Training data accuracy score is : ', training_data_accuracy)

 Training data accuracy score is :  0.9865985576923076


In [61]:
# accuracy score on the test data 
X_test_pred = logist_model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_pred, y_test)

In [62]:
print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the test data :  0.9790865384615385


Build predictive system

In [64]:
X_news = X_test[0] # check the prediction of the 0 means the 1st news or 1st row of our dataset

prediction = logist_model.predict(X_news)
print(prediction)

if prediction[0] == 0:
    print('The news is real.')
else:
    print('The new is Fake')

[1]
The new is Fake


In [65]:
# check if the printed results is correct 
print(y_test[0]) # where y containes the label 

1


by concluding is that the prediction is correct

In [67]:
# Example 2:
print(y_test[3])

0
